In [1]:
import os
from os import environ as ENV
ENV['CUDA_VISIBLE_DEVICES'] = '0'
ROOT = ENV['PWD']

import jax.numpy as jnp
import jax
import numpy as np
import numpy.typing as npt
Float = np.float64

from itertools import combinations
from tqdm.notebook import tqdm

print(jax.devices())
jax.config.update("jax_enable_x64", True)

import pennylane as qml
dev = qml.device("lightning.gpu", wires=18)

[cuda(id=0)]


In [2]:
TESTING = False

In [3]:
# pennylane-catalyst uv/venv metagarbage
import catalyst
# https://github.com/PennyLaneAI/catalyst/pull/1839
# catalyst.utils.runtime_environment.get_cli_path = lambda: ENV['PWD'] + '/.venv/bin/catalyst'
catalyst.utils.runtime_environment.get_cli_path()

'/home/user/work/quadrigems/.venv/bin/catalyst'

In [4]:
# remember to delete cache when rewriting function
import diskcache as dc
cache = dc.Cache(
    ROOT + '/.cache/pennylane_sim/',
    size_limit = 2 ** 32, # 4GB
)

In [5]:
import sys
sys.path.append('..')

from circuit_postprocess import *
from should_be_stdlib import *
from neurodata import *

In [6]:
# if either is the 0 vector, then norm is undefined. unlikely to occur
@qml.qjit(seed=0)
@qml.qnode(dev)
def swap_circuit(data1:npt.NDArray[Float], data2:npt.NDArray[Float], norm:bool=True) -> npt.NDArray[Float]:
    l1, l2 = len(data1), len(data2)
    assert l1 == l2

    # normalize input data
    data1 = data1 / jnp.linalg.norm(data1, ord=1) * jnp.pi/2
    data2 = data2 / jnp.linalg.norm(data2, ord=1) * jnp.pi/2
    # print(data1)
    # print(data2)

    # data embedding
    for i in range(l1 + l2):
        qml.H(i)
    for i in range(l1):
        qml.RY(data1[i], i)
    for i in range(l2):
        qml.RY(data2[i], l1 + i)

    # QFT is inv DFT; inv QFT is DFT
    qml.QFT(wires = range(l1))
    qml.QFT(wires = range(l1, l1 + l2))

    # Swap test
    for i in range(l1):
        qml.CNOT(wires = [i, l1 + i])
    qml.Barrier()
    for i in range(l1):
        qml.H(i)

    return qml.probs(wires = range(l1 + l2))

In [7]:
#TODO: memoize over the neuron idx

from time import time as now

# don't cache while debugging
# @cache.memoize()
def get_fidelity_memo(data1:list[int], data2:list[int]) -> tuple[Float,Float]:
    probs = swap_circuit(data1, data2)
    out = swap_expectation(probs, len(data1))
    return out

def get_fidelity(data1:list[int], data2:list[int]) -> tuple[Float,Float]:
    # data1 should be less than data2
    if is_array_lesser(data2, data1):
        return get_fidelity(data2, data1)
    # normalize data
    data1 = data1 / np.linalg.norm(data1, ord=1) * np.pi/2
    data2 = data2 / np.linalg.norm(data2, ord=1) * np.pi/2
    return get_fidelity_memo(data1, data2)

In [8]:
# should get 1
a = np.array([2,2,2, 2,2,2, 2,2,2,], dtype=Float)
b = np.array([3,3,3, 3,3,3, 3,3,3,], dtype=Float)
#normalization will make these equal to eachother

In [9]:
print(a.dtype)

float64


In [10]:
print('Drawing')
print(qml.draw(swap_circuit)(a, b))

Drawing
 0: ──H──RY(0.17)─╭QFT─╭●──────────────────────────||──H─┤ ╭Probs
 1: ──H──RY(0.17)─├QFT─│──╭●───────────────────────||──H─┤ ├Probs
 2: ──H──RY(0.17)─├QFT─│──│──╭●────────────────────||──H─┤ ├Probs
 3: ──H──RY(0.17)─├QFT─│──│──│──╭●─────────────────||──H─┤ ├Probs
 4: ──H──RY(0.17)─├QFT─│──│──│──│──╭●──────────────||──H─┤ ├Probs
 5: ──H──RY(0.17)─├QFT─│──│──│──│──│──╭●───────────||──H─┤ ├Probs
 6: ──H──RY(0.17)─├QFT─│──│──│──│──│──│──╭●────────||──H─┤ ├Probs
 7: ──H──RY(0.17)─├QFT─│──│──│──│──│──│──│──╭●─────||──H─┤ ├Probs
 8: ──H──RY(0.17)─╰QFT─│──│──│──│──│──│──│──│──╭●──||──H─┤ ├Probs
 9: ──H──RY(0.17)─╭QFT─╰X─│──│──│──│──│──│──│──│───||────┤ ├Probs
10: ──H──RY(0.17)─├QFT────╰X─│──│──│──│──│──│──│───||────┤ ├Probs
11: ──H──RY(0.17)─├QFT───────╰X─│──│──│──│──│──│───||────┤ ├Probs
12: ──H──RY(0.17)─├QFT──────────╰X─│──│──│──│──│───||────┤ ├Probs
13: ──H──RY(0.17)─├QFT─────────────╰X─│──│──│──│───||────┤ ├Probs
14: ──H──RY(0.17)─├QFT────────────────╰X─│──│──│───||────┤ ├Probs
15

In [11]:
exp_swap, fidelity = get_fidelity(a, b)
exp_swap, fidelity

(Array(0.88606496, dtype=float64), Array(0.94303248, dtype=float64))

In [12]:
if TESTING:
    0 / 0

# run simulated results on significant neurons of smallest recording

In [13]:
record = load_record(DEFAULT_RECORD)
sig_neurons = get_sig_neurons(record)
tuning_curves = get_tuning_curves(record).loc[sig_neurons]
coords = get_coords(record).loc[sig_neurons]

In [14]:
try:
    quantum_fidelity = pd.read_csv(ROOT + '/data/quantum_fidelity.csv')

except:
    quantum_fidelity = pd.DataFrame(columns = ['A', 'B', 'f', 'fidelity'])

    # A x B
    pairs, pairs_len = combinations(sig_neurons, 2), (len(sig_neurons) * (len(sig_neurons) - 1) // 2)
    for a, b in tqdm(pairs, total=pairs_len):
        a_i, b_i = tuning_curves.loc[a], tuning_curves.loc[b]
        f, fidelity = get_fidelity(a_i, b_i)
        quantum_fidelity.loc[len(quantum_fidelity)] = [a, b, f, fidelity]
        quantum_fidelity.loc[len(quantum_fidelity)] = [b, a, f, fidelity]

    # A x A = 1 ??? not working
    for a in tqdm(sig_neurons):
        a_i = tuning_curves.loc[a]
        f, fidelity = get_fidelity(a_i, a_i)
        quantum_fidelity.loc[len(quantum_fidelity)] = [a, a, f, fidelity]

    quantum_fidelity.to_csv(ROOT + '/data/quantum_fidelity.csv')


In [15]:
quantum_fidelity_matrix = quantum_fidelity.pivot_table(index = 'A', columns = 'B')['fidelity']

In [16]:
quantum_fidelity_matrix

B,23,32,35,36,37,50,53,55,57,59,...,411,416,419,420,421,424,425,426,428,435
A,,,,,,,,,,,,,,,,,,,,,
23,0.940140,0.934226,0.940871,0.946093,0.940307,0.940192,0.941881,0.940328,0.938246,0.940784,...,0.941057,0.940519,0.940024,0.939312,0.926623,0.937239,0.940583,0.940746,0.939575,0.941619
32,0.934226,0.928966,0.934924,0.940397,0.934481,0.934343,0.936011,0.934490,0.932356,0.934927,...,0.935213,0.934719,0.934431,0.933643,0.920515,0.931490,0.934711,0.934883,0.933594,0.935780
35,0.940871,0.934924,0.941590,0.946788,0.941022,0.940913,0.942596,0.941047,0.938968,0.941507,...,0.941788,0.941233,0.940735,0.940030,0.927362,0.937931,0.941308,0.941468,0.940312,0.942332
36,0.946093,0.940397,0.946788,0.952274,0.946297,0.946138,0.947873,0.946284,0.944138,0.946763,...,0.947053,0.946523,0.946100,0.945386,0.932182,0.943127,0.946538,0.946726,0.945495,0.947633
37,0.940307,0.934481,0.941022,0.946297,0.940486,0.940379,0.942053,0.940523,0.938426,0.940951,...,0.941227,0.940697,0.940256,0.939516,0.926735,0.937473,0.940759,0.940910,0.939703,0.941787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,0.937239,0.931490,0.937931,0.943127,0.937473,0.937313,0.938978,0.937447,0.935388,0.937896,...,0.938154,0.937660,0.937137,0.936508,0.923919,0.934393,0.937663,0.937884,0.936693,0.938737
425,0.940583,0.934711,0.941308,0.946538,0.940759,0.940645,0.942326,0.940779,0.938703,0.941237,...,0.941506,0.940979,0.940513,0.939784,0.927082,0.937663,0.941037,0.941178,0.940019,0.942065
426,0.940746,0.934883,0.941468,0.946726,0.940910,0.940801,0.942487,0.940945,0.938826,0.941386,...,0.941676,0.941113,0.940623,0.939936,0.927098,0.937884,0.941178,0.941380,0.940159,0.942223


In [17]:
quantum_fidelity_matrix[23][32]

np.float64(0.9342262766708186)

# TODO

- memoize functions
- run on GPU
- run on QPU (no mitigation)
- run on QPU (with Mitiq)
- end to end analysis